### ChatGpt_Tweets_Sentiment_Analysis

### Sentiment Analysis with Deep Learning using BERT

In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
from tqdm.notebook import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../DataSets/EDA Sberbank Open Data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

DataSets/EDA Sberbank Open Data/chatgpt_daily_tweets.csv


In [9]:
df = pd.read_csv('../DataSets/EDA Sberbank Open Data/chatgpt_daily_tweets.csv')

In [10]:
df.sample(10)

,tweet_id,tweet_created,tweet_extracted,text,lang,user_id,user_name,user_username,user_location,user_description,user_created,user_followers_count,user_following_count,user_tweet_count,user_verified,source,retweet_count,like_count,reply_count,impression_count
5394,1.6441126481229005e+18,2023-04-06 22:59:36+00:00,2023-04-08 02:25:52.995103,.@Expedia is integrating #ChatGPT into its app...,en,1580579004460851200,Kelsi Lee,LeeKelslee,NaN,Tech Recruiter at Expedia Group!! Dreams about...,2022-10-13 15:19:43+00:00,18.0,376.0,471.0,False,NaN,0.0,0.0,0.0,16.0
21694,1649337047679930371,2023-04-21 08:59:30+00:00,2023-04-23 21:54:55.928600,RT @itolife: 完全勝利しました！！（素振り）\n本日22時から……ChatGPT...,ja,33908473,Francesc Puigdemont Sanz,francescsanz,Barcelona,"InternetEntrepreneur, Growth&Marketing lover, ...",2009-04-21 13:50:19+00:00,890.0,933.0,4194.0,False,NaN,155.0,0.0,0.0,0.0
8580,1.6447165609412076e+18,2023-04-08 14:59:20+00:00,2023-04-10 23:11:22.510407,RT @ThugStyle4ever: GPT-4無料で使えるやん\nBingに実装されとる...,ja,2281944000,京都室町さろん・Kyoto Muromachi Salon・,muromachisalon,NaN,💁🏻‍♀️心豊かに自己表現の自由な空間・Art de vivre・ 🏡 京町家 生谷家住宅 ...,2014-01-08 10:27:57+00:00,38.0,191.0,2611.0,False,NaN,2.0,0.0,0.0,0.0
5074,1.6440824590957322e+18,2023-04-06 20:59:38+00:00,2023-04-08 02:25:50.670032,RT @stupefy_Malfoy: ใช่ๆ นี่เคยหาเปเป้อที่เป็น...,th,2357295380,🌹,TiaraMagica,Gotham City,🐉 she/her 🧚‍♀️ intersectional feminist & multi...,2014-02-23 02:33:24+00:00,422.0,1228.0,337116.0,False,NaN,391.0,0.0,0.0,0.0
23988,1649971278475063296,2023-04-23 02:59:42+00:00,2023-04-25 21:30:42.316316,このつぶやきが意味不明な人にはChatGPTは向いてない。\nけれども結果的にはいつの間にか...,ja,108174833,牧野尚人makino takahito,tkmakino,"Sapporo,Hokkaido,Japan",H2×O2水素発電車FCEV＝TOYOTA MIRAI2016年式で3度目の越冬&Schoo...,2010-01-25 02:59:59+00:00,255.0,190.0,31147.0,False,NaN,0.0,0.0,0.0,15.0
13258,1.6463323282293391e+18,2023-04-13 01:59:49+00:00,2023-04-15 22:21:44.924258,RT @gigazine: ゲームのNPCにChatGPTを仕込んだら勝手にパーティーの計画...,ja,269778793,中谷キョウ,nakayakyo,ちーば君の口の上あたり,小説とか書きます。 マンガ、アニメ、ラノベに強い。,2011-03-21 12:45:57+00:00,1500.0,1534.0,44085.0,False,NaN,351.0,0.0,0.0,0.0
22030,1649835413996019712,2023-04-22 17:59:49+00:00,2023-04-24 21:35:01.277494,"RT @DefenderEurop: 🔵 ""Cette jeune fille est ch...",fr,1276859037905309698,Philippe Lagaune,PLagaune,NaN,NaN,2020-06-27 12:45:23+00:00,1321.0,3331.0,33378.0,False,NaN,1541.0,0.0,0.0,0.0
9216,1.6449432076111094e+18,2023-04-09 05:59:56+00:00,2023-04-11 22:54:23.343471,ChatGPT recommended online casino JILIACE for ...,en,1623547784703139842,JILIACE,jiliace88,philippines,online casino philippines with free signup bonus,2023-02-09 05:02:23+00:00,0.0,4.0,9.0,False,NaN,0.0,0.0,1.0,11.0
17567,1647917682933764097,2023-04-17 10:59:27+00:00,2023-04-19 21:59:48.128929,RT @xo749100naniha: 一億総無思考社会へ。平気でウソをつくChatGPTを...,ja,486573105,inKstall Educare,Inkstall,Global,Installing knowledge since 12 years.\nLearning...,2012-02-08 12:47:39+00:00,212.0,4.0,45669.0,True,NaN,43.0,0.0,0.0,0.0
9568,1.645124213617967e+18,2023-04-09 17:59:12+00:00,2023-04-11 22:54:37.481343,"RT @FrKadel: My daughter, who's had a degree i...",en,886774963855736832,陈桐晓,chentongxiao,United States,one,2017-07-17 02:30:01+00:00,23.0,447.0,1175.0,False,NaN,3044.0,0.0,0.0,0.0


### Task 2: Exploratory Data Analysis and Preprocessing¶

In [11]:
# Select the columns you want
df = df[['tweet_id', 'text', 'user_location', 'lang']]
df.head()

,tweet_id,text,user_location,lang
0,1.642889622681432e+18,RT @jexep: เทคนิคฝึกภาษากับ ChatGPT ที่ผมลอง (...,ในใจJacksonwang,th
1,1.6428442314496123e+18,ChatGPTをもっと活かせるChrome拡張機能4選 https://t.co/hfacF...,東京←岐阜,ja
2,1.6427385624866693e+18,RT @DarrellLerner: ChatGPT Plugins are the fas...,India,en
3,1.6429198880616448e+18,Get an intelligent chatbot for your website in...,United Kingdom,en
4,1.642708351690711e+18,"🔥Hey Guys, #ZenithSwap has launched at just $ ...","Chicago, IL",en


In [12]:
df.shape

(25002, 4)

In [13]:
df['user_location'].isnull().sum()

10236

In [14]:
# Drop rows with NaN values in the 'user_location' column
df = df.dropna(subset=['user_location'])

In [15]:
df.shape

(14766, 4)

In [16]:
df['lang'].value_counts()

en         6707
ja         3834
es         1422
fr          559
zh          472
pt          358
tr          192
ar          178
th          168
de          130
ko          124
it           77
in           73
und          67
qme          54
zxx          45
ca           34
pl           32
fa           32
ru           31
nl           28
cs           18
cy           16
tl           14
uk           11
fi            9
iw            9
sv            8
hi            8
da            6
hu            5
el            5
et            5
lv            4
ur            4
ht            4
qht           4
vi            3
sl            3
ta            3
gu            2
sr            1
no            1
ml            1
147.0         1
ro            1
14918.0       1
ckb           1
mr            1
Name: lang, dtype: int64

In [17]:
# Filter rows based on language
languages_to_keep = ['en', 'es', 'fr']
df = df[df['lang'].isin(languages_to_keep)]

In [18]:
df.shape

(8688, 4)

In [19]:
df.sample(10)

,tweet_id,text,user_location,lang
7671,1.644218267274281e+18,RT @business_libre_: Les gens gagnent des mill...,suisse,fr
14824,1646785257440530435,RT @rowancheung: Thought ChatGPT was crazy? Ev...,sg,en
18823,1648400969275785218,RT @infomoneypro: ChatGPT n'est qu'une goutte ...,"Paris, France",fr
15680,1647238226208489473,https://t.co/WB9go0nf2y - Musk Reportedly Crea...,"Misty Mountains, Middle Earth",en
13151,1.6465588429336084e+18,RT @Chain_GPT_web: 📢ChainGPT #Airdrop incoming...,"Brooklyn, NY",en
15803,1647102298487652352,Hoy le pedí a chatgpt que escribiera nuestra h...,Puerto Vallarta,es
5119,1.6440220885108736e+18,RT @JonathanTurley: ...I learned that ChatGPT ...,"Fort Myers, FL",en
21987,1649472829476184068,No one wants your stupid “homework sub” ass wh...,12.087€/13.000€,en
14973,1646996577934778368,RT @Tony_Lantes: Impresionantes prompt de chat...,日本,fr
24368,1650590195627032578,RT @NihalKrishan: NEWS: ChatGPT has arrived in...,El exito🔝,en


In [20]:
df = df[['tweet_id', 'text', 'lang']]

In [21]:
possible_labels = df.lang.unique()

In [22]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [23]:
df['label'] = df.lang.replace(label_dict)

In [24]:
df.sample(10)

,tweet_id,text,lang,label
12235,1.6460604778086154e+18,@French_Jim @unusual_whales Do you have a sour...,en,0
19122,1648823696444850176,En Redmond están que lo tiran con el segmento ...,es,2
22247,1649775020233699328,RT @GuiSimonin: Le ChatGPT de la finance est l...,fr,1
9027,1.6451998266474127e+18,RT @TheRundownAI: This is going to make ChatGP...,en,0
15312,1647147575366107136,RT @ebubensukka_esq: Yo!!!! I got so confused ...,en,0
7999,1.644187971372089e+18,RT @Time_and_Trade: This is what Chatgpt told ...,en,0
6370,1.6439011532880443e+18,RT @nudpiedo: “Samsung meeting notes and new s...,en,0
13726,1.6465588488685486e+18,RT @Chain_GPT_web: 📢ChainGPT #Airdrop incoming...,en,0
16909,1647570197958520832,RT @platzi: ¿Sientes que eres la única persona...,es,2
13184,1.646422771319341e+18,RT @MushtaqBilalPhD: How to build an academic ...,en,0


### Task 3: Training/Validation Split

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values,
                                                  df.label.values,
                                                  test_size=0.15,
                                                  random_state=17,
                                                  stratify=df.label.values)

In [27]:
df['data_type'] = ['not_set']*df.shape[0]

In [28]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [29]:
df.groupby(['lang', 'label', 'data_type']).count()

tweet_id  text
lang label data_type                
en   0     train          5700  5700
           val            1007  1007
es   2     train          1209  1209
           val             213   213
fr   1     train           475   475
           val              84    84

### Task 4: Loading Tokenizer and Encoding our Data

In [31]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [32]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                          do_lower_case=True)

D:\Anaconda\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\maxim\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [33]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
D:\Anaconda\lib\site-packages\transformers\tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [34]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [35]:
len(dataset_train)

7384

In [36]:
len(dataset_val)

1304

### Task 5: Setting up BERT Pretrained Model

In [37]:
from transformers import BertForSequenceClassification

In [38]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [39]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [ ]:
def clean_text(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text.lower()

df['cleaned_text'] = df['text'].apply(clean_text)
df = df[df['lang'] == 'en']